# Apriori Algorithm : An Iterative Approach

In [3]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
from collections import defaultdict


'''The following function loads the transactions from a file
   Pass as the file name in the function. The function will
   essentially return a list that contains item sets. Notice
   that unique items in each basket is returned. If an item say
   shrimp appears more than once in a particular basket it will be
   return only once for that basket'''

def load_transactions(path_to_file):
    '''Returns a list of transactions and frozenset of items  '''
    
    transaction_list = []
    transaction_item_Set= set()
    
    with open(path_to_file,'r') as f:
        for lines in f:
            str_line = set(lines.strip().split(','))
            for each_item_in_line in str_line:
                transaction_item_Set.add(frozenset([each_item_in_line]))
            transaction_list.append(str_line)
            
    return transaction_list,transaction_item_Set

''' This function will generate first set of candidate pairs.
    Create a dictionary with key and value pairs. The key
    will be the item and the value will be The support count i.e the number of baskets 
    'I' item appears in the dataset. If an item in the basket set
    increment the counts '''

def generateSetC1(Transactions, Transaction_sets):
   
    '''Parameters:
       -------------
       Transactions: list
       Transaction_sets: sets
       
       Pass list of items and set of items
       
       Return:
       --------------
       Method returns a dictionary with the counts, and set of items 
       '''
    
    dictionary = defaultdict()
    _item_Set = set()
    
    for s in Transaction_sets:
        for t in Transactions:
            if s.issubset(t):
                if s in dictionary :
                    dictionary[s] = dictionary[s]+1
                else:
                    dictionary[s]=1


    for key,value in dictionary.items():
        _item_Set.add(key)   
    
            
    return dictionary,_item_Set


'''This function will return dictionary as dataframe so its more easy to interpret visually'''

def convertToDataFrame(dictionary):
    diction_df = pd.DataFrame(data = dictionary.items(), columns = ['Item Set', 'Support Count'])
    return diction_df
    
'''Parameters : C1, minimum_support_count
        C1: C1 should be a dictionary generated from def generateSetC1() 
        minimum_support_count : pass in an integer or a float value
        
        Return : The method will return a dictionary with items whose counts
        are above the threshold value i.e the support count
'''

def L1(C1,minimum_support_count):
        _itemSet = set()
        new_dict =dict()
        
        for item_askey, count_asvalue in C1.items():
            if count_asvalue >= minimum_support_count:
                _itemSet.add(item_askey)
                new_dict[item_askey] = count_asvalue
                
        return new_dict,_itemSet
    
'''  The function  generateAllPossibleSubsets generates all possible subsets of certain size '''   
def generateAllPossibleSubsets(item_set, size_of_subset):
    import itertools
    return([set(i) for i in itertools.combinations(item_set,size_of_subset)])

def unionOfSets(_itemSet,length):
    temp_set = set()
    for i in _itemSet:
        for j in _itemSet:
            if len(i.union(j))==length:
                temp_set.add(i.union(j))
    return temp_set

############################################################
##################### Compute Candidate pairs ##############
###########################################################

'''Performs data base scan and compute the next candidate set CK from LK-1'''

def Ck_generation(Lk_1,transactions,k):
     
    '''Parameters:
       -------------
       Lk-1: set
       transaction: list
       
       Pass transactions as list of items
       Pass Lk-1 i.e frequent item set of length k-1 
       
       Return:
       --------------
       Method returns a dictionary with the counts,set of items and dataframe 
       
    '''
        
    dictionary = defaultdict(int)
    _item_Set  = set()
    
    '''To generate Suppose C2 we need to join L1+L1 to form candiate pairs.
        Note that no purning or deleting of item set is required in the first
        stage'''
    
    C_k_candi_set =unionOfSets(Lk_1,k)
    
    if(k==2):
        for set_s in C_k_candi_set:
            for t in transactions:
                if set_s.issubset(t):
                    if set_s in dictionary :
                        dictionary[set_s] = dictionary[set_s]+1
                    else:
                        dictionary[set_s]=1
                        
        pd_frame_dict = convertToDataFrame(dictionary)

        
    else:
        temp_candi = C_k_candi_set.copy()
        
        '''Based on the Apriori property that all subsets
            of a frequent itemset must also be frequent, we can determine that latter
            candidates cannot possibly be frequent.'''
        
        
        for itemsSets in C_k_candi_set:
            if (has_infrequent_subset_2(itemsSets,Lk_1,k)):
                temp_candi.remove(itemsSets)
            else:
                temp_candi.add(itemsSets)
                
        # Now with the candidate pairs pruned and stored in temp_candi compute
        # the number baskets those items appear 

        for items in temp_candi:
            for temp_candi in transactions:
                if items.issubset(temp_candi):
                    dictionary[items]+=1
                    
        
        pd_frame_dict = convertToDataFrame(dictionary)
            
              
        for key,value in dictionary.items():
            _item_Set.add(frozenset([key]))
        
    
    return dictionary,pd_frame_dict,_item_Set


'''The following function checks if the the item is not frequent itemset
   The function will generate all possible subsets of length k-1.
   All these subsets of k-1 are checked against set containing frequent item set. '''


def has_infrequent_subset_2(Ck_candidate_set, Lk_1_freq_item_set, k):
    
    '''Parameters
    ------------------------------------
        Ck_candidate_set : Set
        Lk_1_freq_item_set: Set
        k : int 
        Pass candidate set and Lk_1 frequent item and the k set as parameter
        k is the size of the subset
        
     --------------------------------------
       Return type : boolean 
       
    '''

    all_k_1_subsets= (generateAllPossibleSubsets(Ck_candidate_set,k-1))
    
    for subset in all_k_1_subsets:
        if(frozenset(subset) not in Lk_1_freq_item_set):
            return True
        return False 

def Lk_generation_with_minimum_Support(Ck_candidate_dict,minimum_support_count):
    _itemSet = set()
    new_dict =dict()
    
    for item_askey, count_asvalue in Ck_candidate_dict.items():
            if count_asvalue >= minimum_support_count:
                _itemSet.add(item_askey)
                new_dict[item_askey] = count_asvalue
                
    
    new_dict_df = convertToDataFrame(new_dict)
    
    return new_dict,new_dict_df,_itemSet

# Generate subsets
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def associationRule(K_item_sets,dict_of_all_counts, minConf):
    rules = []
    for k, itemSet in K_item_sets.items():
        for i in itemSet:
            subsets = powerset(i)
            for s in subsets:
                confidence = float(
                    dict_of_all_counts[i] / dict_of_all_counts[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(i.difference(s)), confidence])
    return rules

def driver_function_apriori(fileName, mini_supp, confidence):
    
    transaction,transact_set = load_transactions(fileName)
   
    # convert percent into an decimal value. 1% of 7501  
    min_support_value = (mini_supp/100)* len(transaction)
    minconfidence     = (confidence/100)
    
    #Create a dictionary that contains all the sets of size 1...n with counts so confidence can be calcuated
    dict_of_all_counts =dict()
    new_Lk = set()
    
    #dictionary to store all frequent itemsets
    K_itemsets = dict()
    
    #list containing all rules
    rules = []

    # Generate C1 candidate set 
    C1_dict , C1_item_set = generateSetC1(transaction, transact_set)
    print('C1 Item Sets')
    display(convertToDataFrame(C1_dict))

    # Using C1 generate L1 candidate set with minimum support 
    L1_dict , L1_item_set = L1(C1_dict,min_support_value)
    print('\nL1 Item Sets')
    L1_df= convertToDataFrame(L1_dict) 
    display(L1_df)
    dict_of_all_counts.update(L1_dict)


    current_LSet = L1_item_set
    
    k = 2

    while(current_LSet):
        K_itemsets[k-1]=current_LSet
        ck_dictionary,pd_dict,ck_itemSet_2 = Ck_generation(current_LSet, transaction,k)
        print(f"\n---------------------------C{k} Item Sets with Given Counts--------------------------------")
        display(pd_dict)
    
        lk_dictionary,df_lk,current_LSet = Lk_generation_with_minimum_Support(ck_dictionary,75.01)
        print(f"\n---------------------------L{k} Item Sets with Minimum Support Counts ---------------------------")
        display(df_lk)
    
        dict_of_all_counts.update(lk_dictionary)
        new_Lk.update(current_LSet)
        k+=1
    
    
    rules = associationRule(K_itemsets,dict_of_all_counts, minconfidence)
    #display rules
    print('\n\n---------------------------------------Mining Rules with Min Confidence---------------------------\n')
    for i in rules :
        print(i)

        

print("-----------------Please enter filename, Minimum Support and Confidence--------------------\n ")
print("Please enter minimum support and confidence level as integer number between 0-100.\n")
print("File Name : ") # enter market_basket.csv
fileName = input()
print(" Enter Minimum Support. Recommendation : As golden rule set support high such as 1% of the total items----\n  ")
print(" Minimum Support : ")
minimum_support = float(input())
print(" Confidence : ")
minimum_confidence = float(input())

driver_function_apriori(fileName,minimum_support,minimum_confidence)

-----------------Please enter filename, Minimum Support and Confidence--------------------
 
Please enter minimum support and confidence level as integer number between 0-100.

File Name : 
market_basket.csv
 Enter Minimum Support. Recommendation : As golden rule set support high such as 1% of the total items----
  
 Minimum Support : 
10
 Confidence : 
30
C1 Item Sets


,Item Set,Support Count
0,(yams),86
1,(mayonnaise),46
2,(gluten free bar),52
3,(candy bars),73
4,(escalope),595
...,...,...
115,(pancakes),713
116,(spaghetti),1306
117,(burgers),654
118,(tomato juice),228



L1 Item Sets


,Item Set,Support Count
0,(green tea),991
1,(eggs),1348
2,(milk),972
3,(chocolate),1229
4,(mineral water),1788
5,(spaghetti),1306
6,(french fries),1282



---------------------------C2 Item Sets with Given Counts--------------------------------


...


---------------------------C4 Item Sets with Given Counts--------------------------------


,Item Set,Support Count
0,"(spaghetti, chocolate, eggs, milk)",22
1,"(spaghetti, mineral water, chocolate, milk)",37
2,"(spaghetti, mineral water, french fries, eggs)",23
3,"(spaghetti, mineral water, french fries, milk)",21
4,"(spaghetti, mineral water, chocolate, eggs)",34
5,"(mineral water, eggs, chocolate, milk)",31
6,"(spaghetti, mineral water, eggs, milk)",33
7,"(spaghetti, mineral water, chocolate, french f...",21



---------------------------L4 Item Sets with Minimum Support Counts ---------------------------


,Item Set,Support Count




---------------------------------------Mining Rules with Min Confidence---------------------------

[{'spaghetti'}, {'mineral water'}, 0.3430321592649311]
[{'chocolate'}, {'mineral water'}, 0.3213995117982099]
[{'milk'}, {'mineral water'}, 0.37037037037037035]
[{'chocolate', 'spaghetti'}, {'mineral water'}, 0.40476190476190477]
[{'chocolate', 'mineral water'}, {'spaghetti'}, 0.3012658227848101]
[{'chocolate', 'eggs'}, {'spaghetti'}, 0.3172690763052209]
[{'spaghetti', 'milk'}, {'mineral water'}, 0.44360902255639095]
[{'milk', 'mineral water'}, {'spaghetti'}, 0.3277777777777778]
[{'chocolate', 'milk'}, {'mineral water'}, 0.43568464730290457]
[{'spaghetti', 'french fries'}, {'mineral water'}, 0.3671497584541063]
[{'french fries', 'mineral water'}, {'spaghetti'}, 0.30039525691699603]
[{'chocolate', 'milk'}, {'spaghetti'}, 0.34024896265560167]
[{'spaghetti', 'milk'}, {'chocolate'}, 0.3082706766917293]
[{'milk', 'eggs'}, {'mineral water'}, 0.42424242424242425]
[{'spaghetti', 'eggs'}, {'min